# 02. Image Model Training
Train CNN on satellite imagery

In [6]:
import sys
sys.path.append('../src')

from preprocessing import load_processed_data
from image_model import train_image_model, predict_with_image_model, load_trained_model
from config import *
import torch

train_df, val_df, test_df, final_test_df = load_processed_data()

## Train Image Model

In [3]:
model, metrics = train_image_model(train_df, val_df)
print(f'Best Val RMSE: ${metrics["val_rmse"]:,.2f}')
print(f'R² Score: {metrics["val_r2"]:.4f}')


TRAINING IMAGE MODEL - TRANSFER LEARNING

⚙️  Configuration:
  Model: efficientnet_b1
  Epochs: 10
  Learning Rate: 0.0003
  Batch Size: 32
  Image Size: 224x224
  Frozen Layers: 80%

🖥️  Device: cpu

📊 Price Normalization:
  Mean:  $  536,458.47
  Std:   $  353,979.35
  Range: $   80,000.00 - $7,700,000.00

📊 Dataset Sizes:
  Training:   11,315 images
  Validation:  2,422 images
  Train batches: 354
  Val batches:   76

🔧 Initializing model...


/Users/yashbansal/Documents/cdc/venv/lib/python3.12/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/Users/yashbansal/Documents/cdc/satellite_property_valuation/notebooks/../src/image_model.py:71: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),



📊 Backbone Parameters:
  Total:        6,513,184
  Frozen:       5,176,784 ( 79.5%)
  Trainable:    1,336,400 ( 20.5%)

📊 Regression Head Parameters: 364,033

📊 Total Trainable: 1,700,433

🚀 Starting training...

Epoch 1/10
--------------------------------------------------------------------------------


Traceback (most recent call last):                 
  File "<string>", line 1, in <module>
  File "/Library/anaconda3/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/anaconda3/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yashbansal/Documents/cdc/venv/lib/python3.12/site-packages/torch/__init__.py", line 426, in <module>
    _load_global_deps()
  File "/Users/yashbansal/Documents/cdc/venv/lib/python3.12/site-packages/torch/__init__.py", line 334, in _load_global_deps
    ctypes.CDLL(global_deps_lib_path, mode=ctypes.RTLD_GLOBAL)
  File "/Library/anaconda3/lib/python3.12/ctypes/__init__.py", line 379, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

## Generate Predictions

In [9]:
# Load trained model from checkpoint
model, metrics = load_trained_model(MODELS_DIR / 'best_image_model.pth')
print(f'Loaded model - Val RMSE: ${metrics["val_rmse"]:,.2f}, R²: {metrics["val_r2"]:.4f}')

# Generate predictions (only for properties with images)
val_ids, val_preds = predict_with_image_model(model, val_df)

# Create a mapping of IDs to predictions
pred_dict = dict(zip(val_ids, val_preds))

# Add predictions to dataframe (NaN for properties without images)
val_df['image_prediction'] = val_df['id'].map(pred_dict)

# Save results
val_df.to_csv('../data/processed/val_with_image_preds.csv', index=False)
print(f'Validation predictions saved! ({len(val_preds)}/{len(val_df)} properties with images)')


📂 Loading model from: /Users/yashbansal/Documents/cdc/satellite_property_valuation/notebooks/../models/best_image_model.pth

📊 Backbone Parameters:
  Total:        6,513,184
  Frozen:       5,176,784 ( 79.5%)
  Trainable:    1,336,400 ( 20.5%)

📊 Regression Head Parameters: 364,033

📊 Total Trainable: 1,700,433
✅ Model loaded successfully
  Epoch: 7
  Val RMSE: $274,948.09
  Val R²: 0.4140
Loaded model - Val RMSE: $274,948.09, R²: 0.4140

🔮 Generating predictions...
  Loaded normalization: mean=$536,458.47, std=$353,979.35
  Total images: 2,422
  Batches: 76


Predicting: 100%|██████████| 76/76 [02:44<00:00,  2.16s/it]

✅ Generated 2,422 predictions
  Price range: $166,282.19 - $2,809,932.93
Validation predictions saved! (2422/2432 properties with images)


✅ Image model trained!
Next: Run `03_tabular_model_training.ipynb`